In [1]:
import numpy as np
import pandas as pd
import datetime

In [4]:
input_data='./input_data/'
output_data='./output_data/'


In [5]:
daily_raw = pd.read_csv(input_data + "chavuma.csv")
daily=daily_raw.dropna().copy()
daily['Date']=pd.to_datetime(daily['Date'],format="%d/%m/%Y")
daily=daily.set_index(pd.DatetimeIndex(daily['Date']))
daily['MonthId']=daily['Year']+daily['Month']/100
daily['WaterMonth']=daily.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)
daily

,Date,Month,Day,Year,WaterYear,Flow,MonthId,WaterMonth
Date,,,,,,,,
1959-10-10,1959-10-10,10,10,1959,1959,49.0,1959.10,1
1959-10-11,1959-10-11,10,11,1959,1959,49.0,1959.10,1
1959-10-12,1959-10-12,10,12,1959,1959,49.0,1959.10,1
1959-10-13,1959-10-13,10,13,1959,1959,49.0,1959.10,1
1959-10-14,1959-10-14,10,14,1959,1959,47.0,1959.10,1
...,...,...,...,...,...,...,...,...
2019-09-26,2019-09-26,9,26,2019,2018,44.0,2019.09,12
2019-09-27,2019-09-27,9,27,2019,2018,43.0,2019.09,12
2019-09-28,2019-09-28,9,28,2019,2018,43.0,2019.09,12


In [ ]:
daily['GapEnd']=daily['Date']!=daily['Date'].shift(1)+pd.DateOffset(1)
daily['GapStart']=np.where(daily['GapEnd']==True,daily['Date'].shift(1),pd.NaT)
daily=daily.astype({'GapStart': 'datetime64'})
daily

In [ ]:
gaps=daily.loc[daily['GapEnd']==True][['GapStart','Flow']].reset_index().rename(columns={'Date':'GapEnd','Flow':'FlowEnd'})
gaps=gaps.drop(0)
gaps.head(2)

In [ ]:
gaps['FlowStart']=gaps.join(daily, on='GapStart',lsuffix='_L',how='inner')['Flow']
gaps['Days']=((gaps['GapEnd']-pd.DateOffset(1))-gaps['GapStart']).dt.days

In [ ]:
gaps.head(2)

In [ ]:
fills=pd.DataFrame()

for row in gaps.itertuples():
    fillset=pd.DataFrame({'GapNo': row.Index, 'Date':pd.date_range(start=row.GapStart+pd.DateOffset(1), end=row.GapEnd-pd.DateOffset(1))})
    joined=fillset.join(gaps,on='GapNo',lsuffix='_L')
    fillset['DayNo']=(fillset['Date']-joined['GapStart']).dt.days
    fillset['Flow']=joined['FlowStart']+((joined['FlowEnd']-joined['FlowStart'])/(joined['Days']+1)*fillset['DayNo'])
    fills=fills.append(fillset)

fills=fills.set_index('Date')
fills

In [ ]:
fills['Year']=fills.index.year
fills['Month']=fills.index.month
fills['Day']=fills.index.day
fills['MonthId']=fills['Year']+fills['Month']/100
fills['WaterYear']=fills.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
fills=fills.drop(['GapNo','DayNo'],axis=1)
daily=daily.drop(['GapStart','GapEnd'],axis=1)
fills

In [ ]:
daily=daily.append(fills)
daily

In [ ]:
monthly=daily.groupby(['MonthId','Year','Month']).size().to_frame(name="DaysRecorded").reset_index(['Month','Year'])
monthly['Days']=monthly.apply(lambda x: datetime.datetime(x['Year'],x['Month'],1),axis=1).dt.daysinmonth
monthly

In [ ]:
monthly.drop(monthly.loc[monthly['DaysRecorded']!=monthly['Days']].index)
monthly=monthly.drop(['DaysRecorded'],axis=1)
monthly['Day']=1
monthly['DateStart']=pd.to_datetime(monthly[['Year','Month','Day']])
monthly=monthly.drop('Day',1)


monthly['WaterYear']=monthly.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
monthly['WaterMonth']=monthly.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)


monthly

In [ ]:
monthly['Flow_min']=daily[['MonthId','Flow']].groupby('MonthId').min()
monthly['Flow_mean']=daily[['MonthId','Flow']].groupby('MonthId').mean()
monthly['Flow_median']=daily[['MonthId','Flow']].groupby('MonthId').median()
monthly['Flow_max']=daily[['MonthId','Flow']].groupby('MonthId').max()
monthly['Flow_range']=monthly['Flow_max']-monthly['Flow_min']
monthly[['Flow_min','Flow_mean','Flow_median','Flow_max','Flow_range']]
monthly

In [ ]:
yearly=monthly[['WaterYear']].groupby('WaterYear').count()

yearly['Flow_min']=daily[['WaterYear','Flow']].groupby('WaterYear').min()
yearly['Flow_median']=daily[['WaterYear','Flow']].groupby('WaterYear').median()
yearly['Flow_mean']=daily[['WaterYear','Flow']].groupby('WaterYear').mean()
yearly['Flow_max']=daily[['WaterYear','Flow']].groupby('WaterYear').max()
yearly['Flow_range']=yearly['Flow_max']-yearly['Flow_min']


yearly

In [ ]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly['Flow_min']=daily[['WaterMonth','Flow']].groupby('WaterMonth').min()
calmonthly['Flow_mean']=daily[['WaterMonth','Flow']].groupby('WaterMonth').mean()
calmonthly['Flow_median']=daily[['WaterMonth','Flow']].groupby('WaterMonth').median()
calmonthly['Flow_max']=daily[['WaterMonth','Flow']].groupby('WaterMonth').max()
calmonthly['Flow_std']=daily[['WaterMonth','Flow']].groupby('WaterMonth').std()
calmonthly['Flow_coefvar']=(calmonthly['Flow_std']/calmonthly['Flow_mean']*100).round(1)


calmonthly

In [ ]:
daily.to_csv(output_data + 'chavuma_daily.csv')
monthly.to_csv(output_data + 'chavuma_monthly.csv')
yearly.to_csv(output_data + 'chavuma_yearly.csv')
calmonthly.to_csv(output_data + 'chavuma_calmonthly.csv')